In [4]:
import os
import random
from openai import OpenAI
import pandas as pd
openai_api_key  = os.getenv('OPENAI_API_KEY')
client=OpenAI()
dataset = pd.read_excel('data_analysis_interview__qa.xlsx')  # Assuming it's an Excel file

In [5]:
def get_user_choices():
    # Define the available categories and difficulties in lowercase
    categories = ['Technical', 'Behavioral']
    difficulties = ['Easy', 'Medium', 'Hard']

    while True:
        chosen_category = input("Enter question's category [Technical, Behavioral]: ").strip().capitalize()
        if chosen_category in categories:
            break
        else:
            print(f"Invalid category. Please choose from {categories}.")

    while True:
        chosen_difficulty = input("Enter question's difficulty [Easy, Medium, Hard]: ").strip().capitalize()
        if chosen_difficulty in difficulties:
            break
        else:
            print(f"Invalid difficulty. Please choose from {difficulties}.")

    return chosen_category.capitalize(), chosen_difficulty

get_user_choices()

('Technical', 'Easy')

In [6]:
def suggest_question(category,difficulty_level):
    filtered_dataset = dataset[(dataset['Category']==category) & (dataset['Difficulty'] == difficulty_level)] 
    single_data_point=filtered_dataset.sample().iloc[0]
    question,answer=single_data_point['Question'],single_data_point['Answer']
    return question,answer
# suggest_question(category='Technical',difficulty_level='Easy')


In [7]:
# Assuming 'dataset' is your DataFrame or data structure

# Create a new question prompt
new_question_prompt = (
    "As a data analysis interviewer, generate a data analysis interview question and provide its answer. "
    "The answer should be at lest 30 words"
    "The output should explicitly include the interviewer's question and the corresponding answer as a dictionary. "
    "The dictionary should also include the difficulty of the question ['Technical', 'Behavioral']. "
    "The dictionary should also include the category of the question ['Easy', 'Medium', 'Hard']. "
    "The dictionary must have only four keys ['Question', 'Answer', 'Category', 'Difficulty'] with the following pattern: "
    "{'Question': generated question, 'Answer': generated answer, 'Category': question's category, 'Difficulty': question's difficulty}.\n"
)

prompt_examples = ""

# Loop through the dataset
for index, row in dataset.iterrows():
    question = row['Question']
    answer = row['Answer']
    category = row['Category']
    difficulty = row['Difficulty']
    prompt_examples += (f"Question: {question}\nAnswer: {answer}\nCategory: {category}\nDifficulty: {difficulty}\n\n")

# Append the new question prompt
prompt_examples += new_question_prompt

# Send the prompt to the GPT-3.5 model
response = client.chat.completions.create(
    model="gpt-3.5-turbo-16k",
    messages=[{'role': 'user', 'content': prompt_examples}],
    max_tokens=150
)

# Extract and print the response content
response_text = response.choices[0].message.content
print(response_text)


{'Question': 'What is the purpose of A/B testing in data analysis?',
 'Answer': 'A/B testing is used to compare two versions of a variable (like a webpage) to determine which one performs better in a controlled experiment.',
 'Category': 'Technical',
 'Difficulty': 'Medium'}


In [8]:
import ast

# Convert the string representation of a dictionary to an actual dictionary
response_dict = ast.literal_eval(response_text)

# Now you can access the data in the dictionary
print(response_dict)  # Output: What is data analysis?


{'Question': 'What is the purpose of A/B testing in data analysis?', 'Answer': 'A/B testing is used to compare two versions of a variable (like a webpage) to determine which one performs better in a controlled experiment.', 'Category': 'Technical', 'Difficulty': 'Medium'}


In [9]:
generated_question=response_dict['Question']
expected_answer=response_dict['Answer']
print(f"Question: {generated_question}")


# Prompt the user to input their answer
user_answer = input("Your Answer: ")

# Optional: Process the user's response
# For example, you can compare it with the generated answer, or store it for later use
print("Your answer was:", user_answer)



Question: What is the purpose of A/B testing in data analysis?
Your answer was: I don't know what is that


In [10]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')
# Function to encode text to vector using SentenceTransformer
def encode(text):
    return model.encode(text, convert_to_tensor=True,device='cuda')
def evaluate_answer(expected_answer,user_answer):
    # Encoding
    expected_answer_vec = encode(expected_answer).unsqueeze(0).cpu()
    user_answer_vec = encode(user_answer).unsqueeze(0).cpu()

    similarity = cosine_similarity(expected_answer_vec, user_answer_vec)

    print(f"Expected Answer: {user_answer}")
    print(f"User's Answer: {expected_answer}")
    print(f"Cosine similarity score: {similarity[0][0]}")



In [11]:
evaluate_answer(expected_answer,user_answer)

Expected Answer: I don't know what is that
User's Answer: A/B testing is used to compare two versions of a variable (like a webpage) to determine which one performs better in a controlled experiment.
Cosine similarity score: 0.13407671451568604


In [21]:
def get_feedback_from_gpt(generated_question, user_answer, expected_answer):
    prompt = (f"Question: {generated_question}\n"
              f"Expected Answer: {expected_answer}\n"
              f"User's Answer: {user_answer}\n\n"
              "Provide detailed feedback on the user's answer for the generated quesion, comparing it with the expected answer and pointing out"
              "any inaccuracies, areas of improvement, or aspects that are well addressed."
              "The output feedback should not be more than 50 words")


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{'role': 'user', 'content': prompt}],
        max_tokens=150
    )

    return response.choices[0].message.content

# Example usage
generated_question = "What is the capital of France?"
user_answer = "The capital of France is Lyon."
expected_answer = "The capital of France is Paris."
feedback = get_feedback_from_gpt(generated_question, user_answer, expected_answer)
print(feedback)


The user's answer is inaccurate. The capital of France is Paris, not Lyon. The user should review their knowledge of French geography to provide correct answers.
